# Methods for `Table`

In [1]:
using JuliaDB

t = loadtable("stocksample", filenamecol = :Ticker, indexcols = [:Ticker, :Date])

Table with 56023 rows, 8 columns:
Columns:
#  colname  type
───────────────────
1  Ticker   String
2  Date     Date
3  Open     Float64
4  High     Float64
5  Low      Float64
6  Close    Float64
7  Volume   Int64
8  OpenInt  Int64

# `select`

`select(table, selection)`

### Note: Multiple selections are "passed around" as NamedTuples

In [8]:
select(t, (:High, :Low) => row -> row.High - row.Low)

56023-element Array{Float64,1}:
  0.01028
  0.0115 
  0.01152
  0.01539
  0.00125
  0.01537
  0.00639
  0.02051
  0.01409
  0.00382
  0.02178
  0.00641
  0.0077 
  ⋮      
  7.03   
  7.93   
  6.53   
 11.77   
 12.349  
 16.06   
 11.12   
  8.49   
  6.47   
  5.59   
  8.16   
  6.51   

# `map`

`map(fun, table; select)`

In [9]:
map(first, t; select = :Ticker) == select(t, :Ticker => first)

true

In [10]:
map(r -> r.High - r.Close, t)

56023-element Array{Float64,1}:
  0.00514
  0.00382
  0.00766
  0.01539
  0.00125
  0.01023
  0.00639
  0.02051
  0.01409
  0.00382
  0.01664
  0.00641
  0.0077 
  ⋮      
  4.06   
  3.72   
  3.7    
  0.42   
 11.529  
  9.43   
  0.16   
  4.72   
  0.45   
  2.58   
  1.47   
  5.37   

# `filter`

`filter(fun, table; selection)` 

In [11]:
# Find days where stock price increased
filter(row -> row.Open < row.Close, t; select = (:Open, :Close))

Table with 26302 rows, 8 columns:
Columns:
#  colname  type
───────────────────
1  Ticker   String
2  Date     Date
3  Open     Float64
4  High     Float64
5  Low      Float64
6  Close    Float64
7  Volume   Int64
8  OpenInt  Int64

# `reduce`

`reduce(reducer, table; select)`

In [12]:
reduce(+, t; select = :Close)

3.9113906714000925e6

In [19]:
using OnlineStats  # more on this later

reduce(Mean(), t; select = :Close)

▦ Series{0}
│ EqualWeight | nobs=56023
└── Mean(69.8176)

# `groupreduce`

`groupreduce(reducer, table, by; selection)`

In [41]:
groupreduce(+, t, :Ticker; select = :Close)

Table with 8 rows, 2 columns:
Ticker          +
─────────────────────────
"aapl.us.txt"   1.86358e5
"amzn.us.txt"   9.36657e5
"dis.us.txt"    2.4894e5
"googl.us.txt"  1.29939e6
"ibm.us.txt"    6.82623e5
"msft.us.txt"   1.51555e5
"nflx.us.txt"   1.26508e5
"tsla.us.txt"   2.7936e5

# `groupby`

`groupby(fun, table [, by]; select, flatten)`

In [42]:
# largest price increase in a single day
groupby(@NT(min_max = extrema), t, :Ticker; select = :Close)

Table with 8 rows, 2 columns:
Ticker          min_max
─────────────────────────────────
"aapl.us.txt"   (0.23051, 175.61)
"amzn.us.txt"   (1.4, 1132.88)
"dis.us.txt"    (0.28298, 118.24)
"googl.us.txt"  (50.005, 1058.29)
"ibm.us.txt"    (3.3901, 186.36)
"msft.us.txt"   (0.0672, 84.56)
"nflx.us.txt"   (1.29, 202.68)
"tsla.us.txt"   (15.8, 385.0)

# `summarize`

`summarize(fun, table, by; select)`

In [26]:
summarize((mean, std), t, :Ticker; select = (:Open, :Close))

Table with 8 rows, 5 columns:
Ticker          Open_mean  Close_mean  Open_std  Close_std
──────────────────────────────────────────────────────────
"aapl.us.txt"   22.2844    22.281      37.7634   37.7645
"amzn.us.txt"   181.747    181.769     239.611   239.548
"dis.us.txt"    20.6162    20.6212     26.4788   26.4787
"googl.us.txt"  389.993    389.856     235.105   235.102
"ibm.us.txt"    48.5355    48.5542     49.271    49.2977
"msft.us.txt"   18.9779    18.9847     16.4161   16.424
"nflx.us.txt"   39.5034    39.5213     47.5678   47.5733
"tsla.us.txt"   150.39     150.355     107.072   107.024

# AoS and SoA

Return the columns (struct of arrays) or rows (array of structs)

`columns(t; selection)` and `rows(t; selection)`

In [43]:
columns(t)

(Ticker = String["aapl.us.txt", "aapl.us.txt", "aapl.us.txt", "aapl.us.txt", "aapl.us.txt", "aapl.us.txt", "aapl.us.txt", "aapl.us.txt", "aapl.us.txt", "aapl.us.txt"  …  "tsla.us.txt", "tsla.us.txt", "tsla.us.txt", "tsla.us.txt", "tsla.us.txt", "tsla.us.txt", "tsla.us.txt", "tsla.us.txt", "tsla.us.txt", "tsla.us.txt"], Date = Date[1984-09-07, 1984-09-10, 1984-09-11, 1984-09-12, 1984-09-13, 1984-09-14, 1984-09-17, 1984-09-18, 1984-09-19, 1984-09-20  …  2017-10-30, 2017-10-31, 2017-11-01, 2017-11-02, 2017-11-03, 2017-11-06, 2017-11-07, 2017-11-08, 2017-11-09, 2017-11-10], Open = [0.42388, 0.42388, 0.42516, 0.42902, 0.43927, 0.44052, 0.45718, 0.45718, 0.44052, 0.43286  …  319.18, 320.23, 332.25, 300.13, 299.5, 307.0, 301.02, 305.5, 302.5, 302.5], High = [0.42902, 0.42516, 0.43668, 0.43157, 0.44052, 0.45589, 0.46357, 0.46103, 0.44566, 0.43668  …  323.78, 331.95, 332.609, 308.69, 306.25, 307.5, 306.5, 306.89, 304.46, 308.36], Low = [0.41874, 0.41366, 0.42516, 0.41618, 0.43927, 0.44052, 0.45

In [44]:
rows(t)

56023-element IndexedTables.Columns{NamedTuples._NT_Ticker_Date_Open_High_Low_Close_Volume_OpenInt{String,Date,Float64,Float64,Float64,Float64,Int64,Int64},NamedTuples._NT_Ticker_Date_Open_High_Low_Close_Volume_OpenInt{Array{String,1},Array{Date,1},Array{Float64,1},Array{Float64,1},Array{Float64,1},Array{Float64,1},Array{Int64,1},Array{Int64,1}}}:
 (Ticker = "aapl.us.txt", Date = 1984-09-07, Open = 0.42388, High = 0.42902, Low = 0.41874, Close = 0.42388, Volume = 23220030, OpenInt = 0)
 (Ticker = "aapl.us.txt", Date = 1984-09-10, Open = 0.42388, High = 0.42516, Low = 0.41366, Close = 0.42134, Volume = 18022532, OpenInt = 0)
 (Ticker = "aapl.us.txt", Date = 1984-09-11, Open = 0.42516, High = 0.43668, Low = 0.42516, Close = 0.42902, Volume = 42498199, OpenInt = 0)
 (Ticker = "aapl.us.txt", Date = 1984-09-12, Open = 0.42902, High = 0.43157, Low = 0.41618, Close = 0.41618, Volume = 37125801, OpenInt = 0)
 (Ticker = "aapl.us.txt", Date = 1984-09-13, Open = 0.43927, High = 0.44052, Low = 0.4